In [1]:
%matplotlib inline 
%load_ext autoreload
%autoreload 2

import os
import copy
import glob
import warnings
import subprocess

import numpy as np
from numpy.lib.recfunctions import append_fields

# Matplotlib related
import matplotlib as mpl
import matplotlib.pyplot as plt

from astropy import units as u
from astropy.table import Table, Column, vstack
from astropy.cosmology import FlatLambdaCDM
from astropy.coordinates import SkyCoord
from astropy.utils.console import ProgressBar

import hsc_massive
from hsc_massive import s16a_path, mass_function
from dsigma.jackknife import add_jackknife_field, add_jackknife_both

envir = s16a_path.set_env(version='astro5')

import palettable
color_fancy = palettable.wesanderson.Mendl_4.mpl_colors
color_bins = palettable.cartocolors.qualitative.Bold_4_r.mpl_colors

### S16A HSC galaxy SMF

In [4]:
um_dir = '/Volumes/astro5/massive/dr16a/um2/s16a_massive_wl/'

mass_prep = np.load(
    os.path.join(um_dir, 's16a_wide2_massive_fsps1_imgsub_use_short_prep.npy'))

In [6]:
rand_dir = os.path.join(envir['s16a'], 'random/hsc_random_with_mask')

rand_prep = np.load(
    os.path.join(rand_dir, 's16a_random_acturus_master.npz'))['random']

In [9]:
mass_tab = Table(mass_prep)
print(mass_tab.colnames)

['object_id', 'ra', 'dec', 'clean_photometry', 'z_spec', 'z', 'z_phot', 'z_type', 'gcmodel_mag', 'gcmodel_mag_err', 'rcmodel_mag', 'rcmodel_mag_err', 'icmodel_mag', 'icmodel_mag_err', 'zcmodel_mag', 'zcmodel_mag_err', 'ycmodel_mag', 'ycmodel_mag_err', 'MSTAR', 'lum_max', 'lum_150', 'lum_120', 'lum_100', 'lum_75', 'lum_50', 'lum_25', 'lum_10', 'lum_5', 'lum_15', 'lum_30', 'lum_40', 'lum_60', 'logm_5', 'logm_10', 'logm_15', 'logm_25', 'logm_30', 'logm_40', 'logm_50', 'logm_60', 'logm_75', 'logm_100', 'logm_120', 'logm_150', 'logm_max', 'photoz_err68_min', 'photoz_err68_max', 'vdisp_sdss', 'vdisp_err_sdss', 'gama_id', 'fluxscale_gama', 'logms_gama', 'logms_err_gama', 'id_cluster_redm', 'p_mem_redm', 'z_lambda_redm', 'z_lambda_err_redm', 'lambda_redm', 'lambda_err_redm', 'p_cen_1_redm', 'id_camira', 'z_camira', 'n_cor_camira', 'logms_camira', 'id_mem_camira', 'w_mem_camira', 'SFR', 'AV', 'TAU', 'AGE', 'ZMETAL', 'r20_max', 'r50_max', 'r80_max', 'r90_max', 'r20_120', 'r50_120', 'r80_120', 'r

In [ ]:
mass_tab.write('..')

In [12]:
# HSC Volume
z_low, z_upp = 0.19, 0.52
s16a_area = 144.7

cosmo = FlatLambdaCDM(H0=70.0, Om0=0.3)
s16a_vol = ((cosmo.comoving_volume(z_upp) - cosmo.comoving_volume(z_low)) * 
            (s16a_area / 41254.0)).value